In [1]:
from pathlib import Path
import pandas as pd 
import tarfile
import urllib.request

def load_housing_data():
    tarball_path = Path('datasets/housing.tgz')
    if not tarball_path.is_file():
        Path('datasets').mkdir(parents=True, exist_ok=True)
        url = 'https://github.com/ageron/data/raw/main/housing.tgz'
        urllib.request.urlretrieve(url=url, filename=tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path='datasets')
    return pd.read_csv(Path('datasets/housing/housing.csv'))

housing = load_housing_data()

In [3]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, )

In [5]:
housing = train_set.drop('median_house_value', axis=1)
housing_labels = train_set['median_house_value'].copy()

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import rbf_kernel

class ClusterSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=10, gamma=1.0):
        self.n_clusters = n_clusters
        self.gamma = gamma
        
    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(self.n_clusters)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self # always return self!
    
    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)
    
    def get_feature_names_out(self, names=None):
        return (f'Cluster {i} similarity' for i in range(self.n_clusters))

In [7]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
import numpy as np

class RatioTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return (X[:, [0]] / X[:, [1]])
    
    def get_feature_names_out(self, input_features=None):
        return ['ratio']
    
class LogTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.log(X)

    def inverse_transform(self, X):
        return np.exp(X)

    def get_feature_names_out(self, input_features=None):
        return [f"log_{f}" for f in input_features]

def ratio_pipeline():
    return make_pipeline(
        SimpleImputer(strategy='median'),
        RatioTransformer(),
        StandardScaler()
    )

log_pipeline = make_pipeline(
    SimpleImputer(strategy='median'),
    LogTransformer(),
    StandardScaler()
)
cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1.)
default_num_pipeline = make_pipeline(SimpleImputer(strategy='median'), StandardScaler())
cat_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(handle_unknown='ignore'))

preprocessing = ColumnTransformer([
    ('bedrooms', ratio_pipeline(), ['total_bedrooms', 'total_rooms']),
    ('rooms_per_house', ratio_pipeline(), ['total_rooms', 'households']),
    ('people_per_house', ratio_pipeline(), ['population', 'households']),
    ('log', log_pipeline, ['total_bedrooms', 'total_rooms', 'population', 'households', 'median_income']),
    ('geo', cluster_simil, ['latitude', 'longitude']),
    ('cat', cat_pipeline, make_column_selector(dtype_include=object)),
],
remainder=default_num_pipeline)

### 1.

*Exercise*: _Try a Support Vector Machine regressor (`sklearn.svm.SVR`) with various hyperparameters, such as `kernel="linear"` (with various values for the `C` hyperparameter) or `kernel="rbf"` (with various values for the `C` and `gamma` hyperparameters). Note that SVMs don't scale well to large datasets, so you should probably train your model on just the first 5,000 instances of the training set and use only 3-fold cross-validation, or else it will take hours. Don't worry about what the hyperparameters mean for now (see the SVM notebook if you're interested). How does the best `SVR` predictor perform?_

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor

param_grid = [
    {'svr__kernel': ['linear'], 
     'svr__C': [10., 100., 300., 1000., 10000.] },
    {'svr__kernel': ['rbf'], 
     'svr__C': [1.0, 3.0, 10., 100., 300., 1000.],
     'svr__gamma': [0.01, 0.03, 0.1, 0.3, 1., 3.]},
]

svr_pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('selector', SelectFromModel(RandomForestRegressor(random_state=42), threshold=0.005)),
    ('svr', SVR())
])

In [19]:
grid_search = GridSearchCV(svr_pipeline, param_grid=param_grid, cv=3, scoring='neg_root_mean_squared_error', verbose=2)
grid_search.fit(housing.iloc[:5000], housing_labels.iloc[:5000])

Fitting 3 folds for each of 41 candidates, totalling 123 fits
[CV] END ....................svr__C=10.0, svr__kernel=linear; total time=   4.5s
[CV] END ....................svr__C=10.0, svr__kernel=linear; total time=   4.4s
[CV] END ....................svr__C=10.0, svr__kernel=linear; total time=   4.6s
[CV] END ...................svr__C=100.0, svr__kernel=linear; total time=   4.5s
[CV] END ...................svr__C=100.0, svr__kernel=linear; total time=   4.5s
[CV] END ...................svr__C=100.0, svr__kernel=linear; total time=   4.5s
[CV] END ...................svr__C=300.0, svr__kernel=linear; total time=   4.5s
[CV] END ...................svr__C=300.0, svr__kernel=linear; total time=   4.5s
[CV] END ...................svr__C=300.0, svr__kernel=linear; total time=   4.5s
[CV] END ..................svr__C=1000.0, svr__kernel=linear; total time=   4.4s
[CV] END ..................svr__C=1000.0, svr__kernel=linear; total time=   4.4s
[CV] END ..................svr__C=1000.0, svr__

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                     SimpleImputer(strategy='median')),
                                                                                    ('standardscaler',
                                                                                     StandardScaler())]),
                                                          transformers=[('bedrooms',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('ratiotransformer',
                                                                                          RatioTransformer()),
                                                                                         ('standardscaler',
                                                                                          Stand...
                                       ('selector',
                                        SelectFromModel(estimator=RandomForestRegressor(random_state=42),
                                                        threshold=0.005)),
                                       ('svr', SVR())]),
             param_grid=[{'svr__C': [10.0, 100.0, 300.0, 1000.0, 10000.0],
                          'svr__kernel': ['linear']},
                         {'svr__C': [1.0, 3.0, 10.0, 100.0, 300.0, 1000.0],
                          'svr__gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0],
                          'svr__kernel': ['rbf']}],
             scoring='neg_root_mean_squared_error', verbose=2)

In [20]:
svr_grid_search_rmse = -grid_search.best_score_
svr_grid_search_rmse

71259.66752562321

In [21]:
grid_search.best_params_

{'svr__C': 10000.0, 'svr__kernel': 'linear'}

### 2.

Exercise: _Try replacing the `GridSearchCV` with a `RandomizedSearchCV`._

In [12]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, loguniform

param_distribs = {
    'svr__kernel': ['linear', 'rbf'],
    'svr__C': loguniform(20, 200_000),
    'svr__gamma': expon(scale=1)
}

svr_pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('svr', SVR())
])

rnd_search = RandomizedSearchCV(
    svr_pipeline, param_distributions=param_distribs, n_iter=50, cv=3, scoring='neg_root_mean_squared_error', random_state=42, verbose=2
)

In [13]:
rnd_search.fit(housing.iloc[:5000], housing_labels.iloc[:5000])

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END svr__C=629.7823295913721, svr__gamma=3.010121430917521, svr__kernel=linear; total time=   0.5s
[CV] END svr__C=629.7823295913721, svr__gamma=3.010121430917521, svr__kernel=linear; total time=   0.4s
[CV] END svr__C=629.7823295913721, svr__gamma=3.010121430917521, svr__kernel=linear; total time=   0.5s
[CV] END svr__C=26290.20646430022, svr__gamma=0.9084469696321253, svr__kernel=rbf; total time=   0.7s
[CV] END svr__C=26290.20646430022, svr__gamma=0.9084469696321253, svr__kernel=rbf; total time=   0.6s
[CV] END svr__C=26290.20646430022, svr__gamma=0.9084469696321253, svr__kernel=rbf; total time=   0.6s
[CV] END svr__C=84.14107900575871, svr__gamma=0.059838768608680676, svr__kernel=rbf; total time=   0.6s
[CV] END svr__C=84.14107900575871, svr__gamma=0.059838768608680676, svr__kernel=rbf; total time=   0.6s
[CV] END svr__C=84.14107900575871, svr__gamma=0.059838768608680676, svr__kernel=rbf; total time=   0.6s
[CV] END

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                           SimpleImputer(strategy='median')),
                                                                                          ('standardscaler',
                                                                                           StandardScaler())]),
                                                                transformers=[('bedrooms',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('ratiotransformer',
                                                                                                RatioTransformer()),
                                                                                               ('standardscaler'...
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x7d7929d819a0>)])),
                                             ('svr', SVR())]),
                   n_iter=50,
                   param_distributions={'svr__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7d792424eb40>,
                                        'svr__gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7d792a35fce0>,
                                        'svr__kernel': ['linear', 'rbf']},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=2)

In [22]:
svr_rnd_search_rmse = -rnd_search.best_score_
pd.Series(svr_rnd_search_rmse).describe()

count        1.000000
mean     57366.722035
std               NaN
min      57366.722035
25%      57366.722035
50%      57366.722035
75%      57366.722035
max      57366.722035
dtype: float64

In [23]:
rnd_search.best_params_

{'svr__C': 157055.10989448498,
 'svr__gamma': 0.26497040005002437,
 'svr__kernel': 'rbf'}

### 3.

Exercise: _Try creating a custom transformer that trains a k-Nearest Neighbors regressor (`sklearn.neighbors.KNeighborsRegressor`) in its `fit()` method, and outputs the model's predictions in its `transform()` method. Then add this feature to the preprocessing pipeline, using latitude and longitude as the inputs to this transformer. This will add a feature in the model that corresponds to the housing median price of the nearest districts._

In [24]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.utils.validation import check_array, check_is_fitted

class LocationClassification(BaseEstimator, TransformerMixin):
    def __init__(self, n_neighbors=3, weights='distance'):
        self.n_neighbors = n_neighbors
        self.weights = weights

    def fit(self, X, y):
        check_array(X)
        self.kneighbors_ = KNeighborsRegressor(n_neighbors=self.n_neighbors, weights=self.weights)
        self.kneighbors_.fit(X=X, y=y)
        self.n_features_in_ = self.kneighbors_.n_features_in_
        return self
    
    def transform(self, X):
        check_is_fitted(self)
        return self.kneighbors_.predict(X).reshape(-1, 1)
    
    def get_feature_names_out(self, names=None):
        check_is_fitted(self)
        n_outputs = getattr(self.kneighbors_, 'n_outputs_', 1)
        class_name = self.kneighbors_.__class__.__name__
        short_name = class_name.lower().replace('_', '')
        return [f'{short_name}_prediction_{i}'
                for i in range(n_outputs)]

In [25]:
from sklearn.utils import estimator_checks
estimator_checks.check_estimator(LocationClassification())

In [26]:
knn_transformer = LocationClassification(n_neighbors=5)
geo_features = housing[['latitude', 'longitude']]
knn_transformer.fit_transform(geo_features, housing_labels)

array([[ 99500.        ],
       [ 91466.66666667],
       [116700.        ],
       ...,
       [ 69640.        ],
       [200000.        ],
       [125966.66666667]])

In [27]:
knn_transformer.get_feature_names_out()

['kneighborsregressor_prediction_0']

In [28]:
from sklearn.base import clone

transformers = [(name, clone(transformer), columns)
                for name, transformer, columns in preprocessing.transformers]
print(transformers)
geo_index = [name for name, _, _ in transformers].index('geo')
print(geo_index)
transformers[geo_index] = ('geo', knn_transformer, ['latitude', 'longitude'])

new_geo_preprocessing = ColumnTransformer(transformers)

[('bedrooms', Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('ratiotransformer', RatioTransformer()),
                ('standardscaler', StandardScaler())]), ['total_bedrooms', 'total_rooms']), ('rooms_per_house', Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('ratiotransformer', RatioTransformer()),
                ('standardscaler', StandardScaler())]), ['total_rooms', 'households']), ('people_per_house', Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('ratiotransformer', RatioTransformer()),
                ('standardscaler', StandardScaler())]), ['population', 'households']), ('log', Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('logtransformer', LogTransformer()),
                ('standardscaler', StandardScaler())]), ['total_bedrooms', 'total_rooms', 'population', 'households', 'median_income']), ('geo', ClusterSimilarity(), 

In [29]:
new_geo_pipeline = Pipeline([
    ('preprocessing', new_geo_preprocessing),
    ('svr', SVR(C=rnd_search.best_params_['svr__C'],
                gamma=rnd_search.best_params_['svr__gamma'],
                kernel=rnd_search.best_params_['svr__kernel']))
])

In [30]:
from sklearn.model_selection import cross_val_score

new_pipe_rmses = -cross_val_score(new_geo_pipeline,
                                  housing.iloc[:5000],
                                  housing_labels.iloc[:5000],
                                  scoring='neg_root_mean_squared_error',
                                  cv=3, verbose=1)
pd.Series(new_pipe_rmses).describe()

count         3.000000
mean     109317.293960
std        2092.133269
min      107244.553529
25%      108261.793315
50%      109279.033100
75%      110353.664175
max      111428.295249
dtype: float64

### 4.

Exercise: _Automatically explore some preparation options using `RandomSearchCV`._

In [ ]:
new_geo_params_distribs = {'preprocessing__geo__n_neighbors': [3, 5, 7],
                   'svr__kernel': ['linear', 'rbf'],
                   'svr__C': loguniform(20, 200_000),
                   'svr__gamma': expon(scale=1)}

new_geo_rnd_search = RandomizedSearchCV(
    new_geo_pipeline, param_distributions=new_geo_params_distribs, n_iter=50, cv=3, 
    scoring='neg_root_mean_squared_error', random_state=42, verbose=2
)

new_geo_rnd_search.fit(housing.iloc[:5000], housing_labels.iloc[:5000])

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END preprocessing__geo__n_neighbors=7, svr__C=30704.49388394695, svr__gamma=0.2026485043181491, svr__kernel=rbf; total time=   0.9s
[CV] END preprocessing__geo__n_neighbors=7, svr__C=30704.49388394695, svr__gamma=0.2026485043181491, svr__kernel=rbf; total time=   0.9s
[CV] END preprocessing__geo__n_neighbors=7, svr__C=30704.49388394695, svr__gamma=0.2026485043181491, svr__kernel=rbf; total time=   0.9s


### 5.

Exercise: _Try to implement the `StandardScalerClone` class again from scratch, then add support for the `inverse_transform()` method: executing `scaler.inverse_transform(scaler.fit_transform(X))` should return an array very close to `X`. Then add support for feature names: set `feature_names_in_` in the `fit()` method if the input is a DataFrame. This attribute should be a NumPy array of column names. Lastly, implement the `get_feature_names_out()` method: it should have one optional `input_features=None` argument. If passed, the method should check that its length matches `n_features_in_`, and it should match `feature_names_in_` if it is defined, then `input_features` should be returned. If `input_features` is `None`, then the method should return `feature_names_in_` if it is defined or `np.array(["x0", "x1", ...])` with length `n_features_in_` otherwise._

In [ ]:
from sklearn.utils.validation import validate_data

class StandardScalerClone(BaseEstimator, TransformerMixin):
    def __init__(self, with_mean=True): # no *args or **kwargs!
        self.with_mean = with_mean

    def fit(self, X, y=None):
        X = validate_data(self, X, ensure_2d=True)
        self.n_features_in_ = X.shape[1]
        if self.with_mean:
            self.mean_ = np.mean(X, axis=0)
        self.scale_ = np.std(X, axis=0, ddof=0)
        self.scale_[self.scale_ == 0] = 1   # Avoid division by zero
        return self
    
    def transform(self, X):
        check_is_fitted(self)
        X = validate_data(self, X, ensure_2d=True, reset=False)
        if self.with_mean:
            X = X - self.mean_
        return X / self.scale_
    
    def inverse_transform(self, X):
        check_is_fitted(self)
        X = validate_data(self, X, ensure_2d=True, reset=False)
        return X * self.scale_ + self.mean_
    
    def get_feature_names_out(self, input_features=None):
        if input_features is None:
            return getattr(self, 'feature_names_in_',
                           [f'x{i}' for i in range(self.n_features_in_)])
        else:
            if len(input_features) != self.n_features_in_:
                raise ValueError('Invalid number of input features')
            if hasattr(self, 'feature_names_in_') and not np.all(
                self.feature_names_in_ == input_features
            ):
                raise ValueError('input_features != feature_names_in_')
            return input_features

In [ ]:
from sklearn.utils.estimator_checks import check_estimator

check_estimator(StandardScalerClone())

In [ ]:
np.random_seed(42)
X = np.random.rand(1000, 3)

scaler = StandardScalerClone()
X_scaled = scaler.fit_transform(X)

assert np.allclose(X_scaled, (X - X.mean(axis=0) / X.std(axis=0)))

In [ ]:
scaler = StandardScalerClone()
X_scaled_uncentered = scaler.fit_transform(X)

assert np.allclose(X_scaled_uncentered, X / X.std(axis=0))

In [ ]:
scaler = StandardScalerClone(X)
X_back = scaler.inverse_transform(scaler.fit_transform(X))

assert np.allclose(X, X_back)

In [ ]:
assert np.all(scaler.get_feature_names_out() == ['x0', 'x1', 'x2'])
assert np.all(scaler.get_feature_names_out(['a', 'b', 'c']) == ['a', 'b', 'c'])

In [ ]:
df = pd.DataFrame({'a': np.random.rand(100), 'b': np.random.rand(100)})
scaler = StandardScalerClone()
X_scaled = scaler.fit_transform(X)

assert np.all(scaler.feature_names_in == ['a', 'b'])
assert np.all(scaler.get_feature_names_out() == ['a', 'b'])1